# Evaluating SETI detection performance - parameter sweeps over multiple raw files
This notebook runs multiple instances of notebook "02_seti_param_sweep.ipynb", which does the following over multiple RAW files:
1) Inputs each RAW file 
2) Runs Rawspec to generate a filterbank .h5 spectrogram file with specified fine FFT size and integration factor n_sti
3) Runs TurboSETI and/or seticore and compiles a list of detections and compute time (wall clock)
4) Repeats the end-to-end runs over a range of specified fine FFT size and integration factor n_sti

This notebook sequences through multiple RAW files.  Parameters are swept and plots are created in the called notebooks.

Note that these sweeps can be done with various branches of seticore or seticore2 and detection SNR or timing comparisons
can be made between branches over multiple runs of this notebook.  For each run of this notebook, 
the desired seticore branch needs to be checked out and compiled, and 
the string "test_case" needs to be set in 01_seti_end_to_end.ipynb to indicate the branch.

In [ ]:
%load_ext autoreload
%autoreload 2

raw_files_are_undefined = False

raw_backup_subdir = 'raw_multichirp/'
if (0):
    raw_file_stem_list = [
        'gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-22.91sec']
elif (0):
    raw_file_stem_list = [
        'gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-366.50sec',
        'meerkat-chirp80-0.0040V-1502.30-1502.70-m10.0-10.0-Hzsec-321.49sec',
        'vlass-chirp80-0.0040V-1401.80-1402.40-m10.0-10.0-Hzsec-8.00sec']
elif(1):
    raw_file_stem_list = [
        'gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-22.91sec',
        'gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-45.81sec',
        'gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-91.63sec',
        'gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-183.25sec']
        # 'gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-366.50sec']
elif (0):
    raw_file_stem_list = [
        'meerkat-chirp80-0.0040V-1502.30-1502.70-m10.0-10.0-Hzsec-321.49sec',
        'meerkat-chirp80-0.0040V-1502.30-1502.70-m10.0-10.0-Hzsec-160.75sec',
        'meerkat-chirp80-0.0040V-1502.30-1502.70-m10.0-10.0-Hzsec-80.37sec']
elif (0):
    raw_file_stem_list = ['meerkat-chirp80-0.0040V-1502.30-1502.70-m10.0-10.0-Hzsec-321.49sec']
elif (0):
    raw_file_stem_list = ['vlass-chirp80-0.0040V-1401.80-1402.40-m10.0-10.0-Hzsec-8.00sec']
elif(0):
    raw_file_stem_list = [
        'gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-183.25sec',
        'gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-366.50sec']
elif (1):
    raw_file_stem_list = ['gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-183.25sec']
elif (0):
    raw_file_stem_list = ['vlass-chirp80-0.0040V-1401.80-1402.40-m10.0-10.0-Hzsec-8.00sec']
elif (0):
    raw_backup_subdir = 'raw_voyager/'
    raw_file_stem_list = ['blc23_guppi_59046_81310_DIAG_VOYAGER-1_0015'] # 23.7 dB, -.504 Hz/sec
elif (0):
    raw_backup_subdir = 'raw_voyager/'
    raw_file_stem_list = ['blc3_guppi_57386_VOYAGER1_0004'] 
elif (0):
    raw_backup_subdir = 'raw_gbt_57388_HIP/'
    raw_file_stem_list = ['blc1_guppi_57388_HIP113357_0010',
                          'blc2_guppi_57388_HIP113357_0010',
                          'blc3_guppi_57388_HIP113357_0010',
                          'blc4_guppi_57388_HIP113357_0010',
                          'blc5_guppi_57388_HIP113357_0010',
                          'blc6_guppi_57388_HIP113357_0010'] 
elif (0):
    raw_backup_subdir = 'raw_gbt_57388_HIP/'
    raw_file_stem_list = ['blc1_guppi_57388_HIP113357_0012',
                          'blc2_guppi_57388_HIP113357_0012',
                          'blc3_guppi_57388_HIP113357_0012',
                          'blc4_guppi_57388_HIP113357_0012',
                          'blc5_guppi_57388_HIP113357_0012',
                          'blc6_guppi_57388_HIP113357_0012'] 
elif (0):
    raw_backup_subdir = 'raw_gbt_59103_HIP/'
    raw_file_stem_list = ['blc40_guppi_59103_03394_DIAG_HIP95631_0015',
                          'blc41_guppi_59103_03394_DIAG_HIP95631_0015',
                          'blc42_guppi_59103_03394_DIAG_HIP95631_0015',
                          'blc43_guppi_59103_03394_DIAG_HIP95631_0015',
                          'blc44_guppi_59103_03394_DIAG_HIP95631_0015',
                          'blc45_guppi_59103_03394_DIAG_HIP95631_0015',
                          'blc46_guppi_59103_03394_DIAG_HIP95631_0015',
                          'blc47_guppi_59103_03394_DIAG_HIP95631_0015'] 
elif (1):
    raw_backup_subdir = 'raw_gbt_59103_Kepler/'
    raw_file_stem_list = ['blc40_guppi_59103_01984_DIAG_KEPLER-160_0010',
                          'blc41_guppi_59103_01984_DIAG_KEPLER-160_0010',
                          'blc42_guppi_59103_01984_DIAG_KEPLER-160_0010',
                          'blc43_guppi_59103_01984_DIAG_KEPLER-160_0010',
                          'blc44_guppi_59103_01984_DIAG_KEPLER-160_0010',
                          'blc45_guppi_59103_01984_DIAG_KEPLER-160_0010',
                          'blc46_guppi_59103_01984_DIAG_KEPLER-160_0010',
                          'blc47_guppi_59103_01984_DIAG_KEPLER-160_0010'] 
elif (1):
    raw_backup_subdir = 'raw_gbt_59103_TIC/'
    raw_file_stem_list = ['blc40_guppi_59103_03721_DIAG_TIC159107668_0016',
                          'blc41_guppi_59103_03721_DIAG_TIC159107668_0016',
                          'blc42_guppi_59103_03721_DIAG_TIC159107668_0016',
                          'blc43_guppi_59103_03721_DIAG_TIC159107668_0016',
                          'blc44_guppi_59103_03721_DIAG_TIC159107668_0016',
                          'blc45_guppi_59103_03721_DIAG_TIC159107668_0016',
                          'blc46_guppi_59103_03721_DIAG_TIC159107668_0016',
                          'blc47_guppi_59103_03721_DIAG_TIC159107668_0016'] 

plot_spectra = False
plot_snr_vs_f = False
zoom_bw_MHz = 1.0
do_profile = False


#### Run end-to-end test over parameter sweeps

In [ ]:
n_raw_file = len(raw_file_stem_list)


for i_raw in range(n_raw_file):

    raw_file_stem = raw_file_stem_list[i_raw]
    print(f'\n\nRaw file {i_raw+1} of {n_raw_file}: {raw_file_stem}\n\n')

    try:      
        %run ./02_seti_param_sweep.ipynb
    except:
        print(f'\n\nError Occurred for Raw file {raw_file_stem}, skip\n\n')

    if (1):
        # delete raw file to limit local file storage
        if (i_raw+1<n_raw_file):
            try:
                print('Deleting ',raw_file_delete_spec)
                os.system('rm '+raw_file_delete_spec)
            except:
                print(' ')

            

In [ ]:
import os

print('All Runs Done')

# Beep 2 times in WSL
os.system("powershell.exe '[console]::beep(261.6,700)'")
os.system("powershell.exe '[console]::beep(261.6,700)'")
